In [133]:
from pandas import read_csv, Series, DataFrame, concat
import numpy as np
import pathlib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

In [92]:
#Number of recommendations set to 20
no_recom = 20
games_data = read_csv('processed_games_for_content-based.csv')
users_data = read_csv('Content_Training_Testing/content_user_train.csv')
#Getting reviews information from the csv generated
games_review = read_csv('steam_games_review.csv', usecols=["name", "positive_review_percent"])
#Creating reverse map of indices and game names
games_indices = Series(games_data.index, index=games_data["name"]).drop_duplicates()
#Getting list of all unique games we have in games dataset
games_list = games_data['name'].unique()
#Creating new dataframe for recommendation
df_rec = list(map(str, range(1, no_recom+1)))
df_new = list(map(str, range(1, no_recom+1)))
df_rec = ["user_id"] + df_rec
df_new = ["user_id"] + df_new

In [160]:
#Total unique games in training dataset
users_data.game.nunique()

4884

In [93]:
#Recommendation functions
#Function taking game titles (game name) and cosine similarity as input arguments and returns most similar games
def make_recommendation_by_sim(game_title, cosine_sim):
    if game_title not in games_list:
        return []
    #Index of the game that matches the name
    id = games_indices[game_title]
    if type(id) is Series:
        return []
    #Calculating pairwise similarity scores of all games with the game mentioned
    similarity_score = list(enumerate(cosine_sim[id]))
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    #Getting similarity scores of top 20 most similar games
    similarity_score = similarity_score[1:no_recom+1]
    #Getting game indices of the above 20 games with matching scores
    games_id =  [i[0] for i in similarity_score]
    #finally returning top most similar games
    return games_data['name'].iloc[games_id].tolist()


In [94]:
#Function taking user id , games user have and the games user already have as input 
#and making recommendations
def make_recommendation_for_user(user_id, game_list, games_of_user):
    if type(game_list) is not list or len(game_list) == 0:
        return DataFrame(data=[[user_id]+[""] * no_recom], columns=df_rec)
    recom_reviews = games_review.loc[games_review['name'].isin(game_list)]
    recom_reviews = recom_reviews.loc[~recom_reviews['name'].isin(games_of_user)]
    recom_reviews = recom_reviews.sort_values(by='positive_review_percent', ascending=False)
    if len(recom_reviews) < no_recom:
        return DataFrame(data=[[user_id] + recom_reviews["name"].tolist() + [""] * (no_recom - len(recom_reviews.index))], columns=df_rec)
    else:
        return DataFrame(data=[[user_id] + recom_reviews["name"].tolist()[0:no_recom]], columns=df_rec)
    

In [210]:
#Function to generate recommendation for a user by a column feature
def make_recommendation_bycolumn(column_name, user_id):
    recom_by_userdata = DataFrame(columns=df_rec)
    games_data[column_name] = games_data[column_name].fillna('')
    #Calculating Cosine Similarity Matrix using the column feature
    count =  CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(games_data[column_name])
    similarity_matrix = cosine_similarity(count_matrix, count_matrix)
    #print(similarity_matrix)
    id = ''
    suggestion_list = list()
    games_user_has_list = list()
    for i, row in users_data.iterrows():
       if row["user_id"] == user_id:   
         recom_by_userdata = concat([recom_by_userdata, make_recommendation_for_user(user_id, suggestion_list, games_user_has_list)])  
         games_user_has_list.extend([row["game"]])
         suggestion_list.extend(make_recommendation_by_sim(row["game"], similarity_matrix))
         break
    recom_by_userdata = concat([recom_by_userdata, make_recommendation_for_user(user_id, suggestion_list, games_user_has_list)], ignore_index=True)
    return recom_by_userdata

In [211]:
df_1 = make_recommendation_bycolumn('popular_tags', 5250)
df_1

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,5250,,,,,,,,,,...,,,,,,,,,,
1,5250,Killing Floor,Team Fortress 2,Warframe,Arma 3,Earth Defense Force: Insect Armageddon,Natural Selection 2,Tom Clancy's Rainbow Six® Vegas 2,Halo: Spartan Strike,Resident Evil Revelations / Biohazard Revelations,...,Primal Carnage,Alien Breed 2: Assault,Defiance,ORION: Prelude,Alien Breed: Impact,Alien Breed 3: Descent,Primal Carnage: Extinction,God Mode,Scourge: Outbreak,Dark Void™


In [167]:
make_recommendation_bycolumn('genre', 5250)

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,5250,,,,,,,,,,...,,,,,,,,,,
1,5250,Portal,The Binding of Isaac: Rebirth,Left 4 Dead 2,Psychonauts,Ori and the Blind Forest,Counter-Strike: Source,BioShock Infinite,Mafia II,Vampire: The Masquerade - Bloodlines,...,Call of Duty® 4: Modern Warfare®,Killing Floor 2,Call of Duty®: Modern Warfare® 2,Wolfenstein: The New Order,Call of Duty®: Black Ops II,Tom Clancy's Rainbow Six® Siege,The Evil Within,Metro: Last Light Redux,Metro 2033 Redux,Call of Duty®: Modern Warfare® 3


In [168]:
make_recommendation_bycolumn('genre_publisher_developer', 5250)

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,5250,,,,,,,,,,...,,,,,,,,,,
1,5250,Portal 2,Left 4 Dead 2,Half-Life 2,Counter-Strike,Counter-Strike: Source,Half-Life,Half-Life 2: Episode Two,Half-Life: Opposing Force,Left 4 Dead,...,Half-Life: Blue Shift,Day of Defeat: Source,Half-Life 2: Deathmatch,Day of Defeat,Team Fortress Classic,Deathmatch Classic,Half-Life: Source,Ricochet,Half-Life Deathmatch: Source,


In [169]:
make_recommendation_bycolumn('genre_popular_tags_developer', 5250)

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,5250,,,,,,,,,,...,,,,,,,,,,
1,5250,Killing Floor,Left 4 Dead 2,Dead Space™ 2,Team Fortress 2,Synergy,Warframe,Arma 3,Earth Defense Force: Insect Armageddon,Tom Clancy's Rainbow Six® Vegas 2,...,Halo: Spartan Strike,Call of Duty®: Black Ops II,Tom Clancy’s Splinter Cell Blacklist,Alien Breed 2: Assault,Primal Carnage,ORION: Prelude,Defiance,Primal Carnage: Extinction,God Mode,Scourge: Outbreak


In [186]:
#Function to generate recommendation for 20 users by a column feature
def make_recommendationforNusers(column_name):
    recom_by_userdata = DataFrame(columns=df_new)
    games_data[column_name] = games_data[column_name].fillna('')
    #Calculating Cosine Similarity Matrix using the column feature
    count =  CountVectorizer(stop_words='english')
    count_matrix = count.fit_transform(games_data[column_name])
    similarity_matrix = cosine_similarity(count_matrix, count_matrix)
    id = ''
    count = 0
    suggestion_list = list()
    games_user_has_list = list()
    for i, row in users_data.iterrows():
       if count == 20:
        break
       if row["user_id"] != id:   
         recom_by_userdata = concat([recom_by_userdata, make_recommendation_for_user(id, suggestion_list, games_user_has_list)])  
         id = row["user_id"]
         count += 1
         suggestion_list = list()
         games_user_has_list = list()
       games_user_has_list.extend([row["game"]])
       suggestion_list.extend(make_recommendation_by_sim(row["game"], similarity_matrix))
    recom_by_userdata = concat([recom_by_userdata, make_recommendation_for_user(id, suggestion_list, games_user_has_list)], ignore_index=True)
    return recom_by_userdata, similarity_matrix

In [191]:
df_genre, genre_matrix = make_recommendationforNusers('genre')
df_genre

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,,,,,,,,,,...,,,,,,,,,,
1,5250.0,Portal,The Binding of Isaac: Rebirth,Ori and the Blind Forest,Left 4 Dead 2,Dishonored,Psychonauts,BioShock Infinite,Counter-Strike: Source,Mafia II,...,Call of Duty: World at War,Echoes+,Synergy,Rexaura,Saints Row IV,Warframe,Call of Duty® 4: Modern Warfare®,Half-Life 2: Update,Fortress Forever,Call of Duty®: Modern Warfare® 2
2,76767.0,Portal,Primordia,The Room,The Binding of Isaac: Rebirth,Left 4 Dead 2,Dishonored,FTL: Faster Than Light,Supreme Commander: Forged Alliance,Psychonauts,...,"Cook, Serve, Delicious!",Machinarium,Mini Metro,"Warhammer® 40,000: Dawn of War® - Game of the ...",Sid Meier's Civilization® V,Counter-Strike: Source,Creeper World 3: Arc Eternal,"Papers, Please",BioShock Infinite,Antichamber
3,86540.0,Portal 2,Grim Legends 2: Song of the Dark Swan,The Witcher® 3: Wild Hunt,Freddi Fish 4: The Case of the Hogfish Rustler...,Hotline Miami,Super Hexagon,The Binding of Isaac: Rebirth,Mount & Blade: Warband,VVVVVV,...,Freddi Fish 2: The Case of the Haunted Schoolh...,Psychonauts,Dishonored,Ori and the Blind Forest,Duck Game,Fallout: New Vegas,BioShock Infinite,Skullgirls,"Cook, Serve, Delicious!",Counter-Strike: Source
4,103360.0,Portal,The Binding of Isaac: Rebirth,Left 4 Dead 2,Psychonauts,Ori and the Blind Forest,Counter-Strike: Source,BioShock Infinite,Mafia II,Vampire: The Masquerade - Bloodlines,...,Call of Duty® 4: Modern Warfare®,Killing Floor 2,Call of Duty®: Modern Warfare® 2,Wolfenstein: The New Order,Call of Duty®: Black Ops II,Tom Clancy's Rainbow Six® Siege,The Evil Within,Metro: Last Light Redux,Metro 2033 Redux,Call of Duty®: Modern Warfare® 3
5,144736.0,Portal,The Binding of Isaac: Rebirth,Left 4 Dead 2,Psychonauts,Ori and the Blind Forest,Counter-Strike: Source,BioShock Infinite,Mafia II,Vampire: The Masquerade - Bloodlines,...,Call of Duty® 4: Modern Warfare®,Killing Floor 2,Call of Duty®: Modern Warfare® 2,Wolfenstein: The New Order,Call of Duty®: Black Ops II,Tom Clancy's Rainbow Six® Siege,The Evil Within,Metro: Last Light Redux,Metro 2033 Redux,Call of Duty®: Modern Warfare® 3
6,181212.0,Portal,The Binding of Isaac: Rebirth,Left 4 Dead 2,Psychonauts,Ori and the Blind Forest,Counter-Strike: Source,BioShock Infinite,Mafia II,Vampire: The Masquerade - Bloodlines,...,Call of Duty® 4: Modern Warfare®,Killing Floor 2,Call of Duty®: Modern Warfare® 2,Wolfenstein: The New Order,Call of Duty®: Black Ops II,Tom Clancy's Rainbow Six® Siege,The Evil Within,Metro: Last Light Redux,Metro 2033 Redux,Call of Duty®: Modern Warfare® 3
7,229911.0,Portal,The Binding of Isaac: Rebirth,Ori and the Blind Forest,Supreme Commander: Forged Alliance,Left 4 Dead 2,Psychonauts,Counter-Strike: Source,BioShock Infinite,"Warhammer® 40,000: Dawn of War® - Game of the ...",...,"Warhammer® 40,000: Dawn of War® - Soulstorm",XCOM: Enemy Unknown,Vampire: The Masquerade - Bloodlines,Mafia II,Echoes+,Synergy,Call of Duty: World at War,Rexaura,Warframe,Panzer Corps
8,298950.0,Portal 2,fault - milestone two side:above,Castle Crashers - Pink Knight Pack,Keep Talking and Nobody Explodes,BattleBlock Theater®,Primordia,Super Hexagon,Mount & Blade: Warband,Castle Crashers - Blacksmith Pack,...,The Binding of Isaac: Rebirth,TIS-100,The Witcher® 3: Wild Hunt,Hotline Miami,The Room,The Expendabros,Choice of Robots,Borderlands 2,GemCraft - Chasing Shadows,Orcs Must Die!
9,299153.0,Portal,The Binding of Isaac: Rebirth,Left 4 Dead 2,Psychonauts,Ori and the Blind Forest,Counter-Strike: Source,BioShock Infinite,Mafia II,Vampire: The Masquerade - Bloodlines,...,Call of Duty® 4: Modern Warfare®,Killing Floor 2,Call of Duty®: Modern Warfare® 2,Wolfenstein: The New Order,Call of Duty®: Black Ops II,Tom Clancy's Rainbow Six® Siege,The Evil Within,Metro: Last Light Redux,Metro 2033 Redux,Call of Duty®: Modern Warfare® 3


In [192]:
print(genre_matrix)

[[1.         0.56694671 0.75592895 ... 0.         0.         0.2236068 ]
 [0.56694671 1.         0.85714286 ... 0.21821789 0.37796447 0.16903085]
 [0.75592895 0.85714286 1.         ... 0.         0.         0.16903085]
 ...
 [0.         0.21821789 0.         ... 1.         0.57735027 0.        ]
 [0.         0.37796447 0.         ... 0.57735027 1.         0.        ]
 [0.2236068  0.16903085 0.16903085 ... 0.         0.         1.        ]]


In [194]:
df_tags, tag_matrix = make_recommendationforNusers('popular_tags')
df_tags

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,,,,,,,,,,...,,,,,,,,,,
1,5250.0,Battlefield: Bad Company 2 Vietnam,Portal,BattleBlock Theater®,Unreal Tournament: Game of the Year Edition,Serious Sam Classic: The Second Encounter,Killing Floor,Descent,Half-Life 2: Episode Two,Counter-Strike: Source,...,Quake III Arena,Shadow Warrior Classic (1997),System Shock 2,Half-Life: Opposing Force,Left 4 Dead,BioShock™,Serious Sam HD: The First Encounter,Black Mesa,Half-Life 2: Episode One,Sanctum 2
2,76767.0,The Witcher 3: Wild Hunt - Expansion Pass,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,BattleBlock Theater®,The Witcher® 3: Wild Hunt,Serious Sam Classic: The Second Encounter,Dishonored,Descent,Killing Floor,...,Counter-Strike: Source,Half-Life 2: Episode Two,QUAKE,Quake III Arena,System Shock 2,Half-Life: Opposing Force,Thomas Was Alone,Shadow Warrior Classic (1997),Stronghold HD,Team Fortress 2
3,86540.0,Battlefield: Bad Company 2 Vietnam,The Witcher® 3: Wild Hunt,The Expendabros,Enigmatis 2: The Mists of Ravenwood,Unreal Tournament: Game of the Year Edition,Botanicula,Half-Life 2,The Journey Down: Chapter Two,Descent,...,Killing Floor,Broforce,Half-Life 2: Episode Two,QUAKE,Amnesia: The Dark Descent,Tomb Raider,Counter-Strike: Source,Quake III Arena,DARK SOULS™ II - Season Pass,Guacamelee! Super Turbo Championship Edition
4,103360.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Killing Floor,Serious Sam Classic: The Second Encounter,Descent,Half-Life 2,Quake III Arena,Half-Life 2: Episode Two,QUAKE,...,System Shock 2,Left 4 Dead,Alien Swarm,Shadow Warrior Classic (1997),Half-Life: Opposing Force,Half-Life 2: Episode One,Team Fortress 2,Serious Sam HD: The First Encounter,QUAKE II,Descent 2
5,144736.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Half-Life 2,Serious Sam Classic: The Second Encounter,Killing Floor,QUAKE,Quake III Arena,Counter-Strike: Source,Half-Life 2: Episode Two,...,Half-Life: Opposing Force,Left 4 Dead,Alien Swarm,Serious Sam HD: The First Encounter,Half-Life 2: Episode One,Team Fortress 2,Descent 2,QUAKE II,BioShock Infinite - Season Pass,Tom Clancy's Ghost Recon®
6,181212.0,Battlefield: Bad Company 2 Vietnam,Portal,Unreal Tournament: Game of the Year Edition,Descent,Serious Sam Classic: The Second Encounter,Killing Floor,Half-Life 2: Episode Two,QUAKE,Quake III Arena,...,Left 4 Dead,BioShock™,Shadow Warrior Classic (1997),Alien Swarm,Half-Life: Opposing Force,Black Mesa,Serious Sam HD: The First Encounter,Half-Life 2: Episode One,Team Fortress 2,QUAKE II
7,229911.0,Battlefield: Bad Company 2 Vietnam,Portal,Unreal Tournament: Game of the Year Edition,Descent,Serious Sam Classic: The Second Encounter,Killing Floor,Half-Life 2: Episode Two,QUAKE,Counter-Strike: Source,...,Left 4 Dead,BioShock™,Half-Life: Opposing Force,Shadow Warrior Classic (1997),Mount Your Friends,Alien Swarm,System Shock 2,Worms Armageddon,Black Mesa,Serious Sam HD: The First Encounter
8,298950.0,The Witcher 3: Wild Hunt - Expansion Pass,Portal 2,Tetrobot and Co.,Battlefield: Bad Company 2 Vietnam,The Binding of Isaac: Rebirth,Escape Goat,Eternal Senia,planetarian ~the reverie of a little planet~,The Witcher® 3: Wild Hunt,...,Mount & Blade: Warband,Epic Battle Fantasy 4,BattleBlock Theater®,Enigmatis 2: The Mists of Ravenwood,Unreal Tournament: Game of the Year Edition,The Expendabros,Outlast: Whistleblower DLC,Primordia,Ori and the Blind Forest,Don't Starve
9,299153.0,Battlefield: Bad Company 2 Vietnam,Portal,Unreal Tournament: Game of the Year Edition,Serious Sam Classic: The Second Encounter,Killing Floor,Counter-Strike: Source,QUAKE,Quake III Arena,Half-Life 2: Episode Two,...,Alien Swarm,Shadow Warrior Classic (1997),Half-Life: Opposing Force,BioShock™,Serious Sam HD: The First Encounter,Half-Life 2: Episode One,Black Mesa,Team Fortress 2,Descent 2,BioShock Infinite - Season Pass


In [195]:
print(tag_matrix)

[[1.         0.40022241 0.38100038 ... 0.17038855 0.10369517 0.14664712]
 [0.40022241 1.         0.52522573 ... 0.11744404 0.10721125 0.15161961]
 [0.38100038 0.52522573 1.         ... 0.02795085 0.         0.14433757]
 ...
 [0.17038855 0.11744404 0.02795085 ... 1.         0.27386128 0.        ]
 [0.10369517 0.10721125 0.         ... 0.27386128 1.         0.        ]
 [0.14664712 0.15161961 0.14433757 ... 0.         0.         1.        ]]


In [196]:
df_gen_pub_dev, gen_pub_dev_matrix = make_recommendationforNusers('genre_publisher_developer')
df_gen_pub_dev

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,,,,,,,,,,...,,,,,,,,,,
1,5250.0,Portal,Left 4 Dead 2,Counter-Strike: Source,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Half-Life 2: Episode One,Synergy,Echoes+,...,Half-Life: Blue Shift,Vanguard Princess Director's Cut,Time Clickers,Day of Defeat: Source,Counter-Strike: Global Offensive,Half-Life 2: Deathmatch,Dota 2,Brawlhalla,Apotheon Arena,Codename CURE
2,76767.0,Portal,Half-Life 2,The Journey Down: Chapter Two,Euro Truck Simulator 2,Left 4 Dead 2,Quake III Arena,Counter-Strike: Source,Half-Life 2: Episode Two,Tomb Raider,...,Half-Life: Opposing Force,Left 4 Dead,XCOM: Enemy Unknown,LIMBO,Euro Truck Simulator 2 - Ice Cold Paint Jobs Pack,Mafia II,Team Fortress 2,Half-Life 2: Episode One,Alien: Isolation,Ittle Dew
3,86540.0,Portal 2,Sam & Max 102: Situation: Comedy,The Wolf Among Us,Sam & Max 105: Reality 2.0,Hotline Miami,Mount & Blade: Warband,The Room,Half-Life 2,Broforce,...,Downwell,GemCraft - Chasing Shadows,Orcs Must Die!,Sam & Max 203: Night of the Raving Dead,Half-Life 2: Episode Two,Sid Meier's Civilization® V,Sid Meier's Civilization V: Brave New World,BioShock Infinite,Counter-Strike: Source,The Talos Principle
4,103360.0,Portal 2,Left 4 Dead 2,Half-Life 2,Counter-Strike: Source,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Alien Swarm,Half-Life 2: Episode One,...,Day of Defeat: Source,Half-Life 2: Deathmatch,Half-Life: Source,Half-Life Deathmatch: Source,,,,,,
5,144736.0,Portal 2,Left 4 Dead 2,Half-Life 2,Counter-Strike: Source,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Alien Swarm,Half-Life 2: Episode One,...,Day of Defeat: Source,Half-Life 2: Deathmatch,Half-Life: Source,Ricochet,Half-Life Deathmatch: Source,,,,,
6,181212.0,Portal 2,Left 4 Dead 2,Counter-Strike,Counter-Strike: Source,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Alien Swarm,Half-Life 2: Episode One,...,Day of Defeat: Source,Half-Life 2: Deathmatch,Half-Life: Source,Half-Life Deathmatch: Source,,,,,,
7,229911.0,Portal 2,Left 4 Dead 2,Counter-Strike: Source,Half-Life 2: Episode Two,Half-Life: Opposing Force,Alien Swarm,Left 4 Dead,Half-Life 2: Episode One,Worms Armageddon,...,Synergy,Chronology,Half-Life: Blue Shift,Vanguard Princess Director's Cut,The Escapists,Super Crate Box,Day of Defeat: Source,Time Clickers,METAL GEAR SOLID V: THE PHANTOM PAIN,Counter-Strike: Global Offensive
8,298950.0,Ori and the Blind Forest (Original Soundtrack),Keep Talking and Nobody Explodes,Portal 2,One Finger Death Punch,Battlefield: Bad Company 2 Vietnam,Sam & Max 102: Situation: Comedy,TIS-100,South Park™: The Stick of Truth™,Luxor Evolved,...,Mount & Blade: Warband,Hotline Miami,Sam & Max 105: Reality 2.0,Unreal Tournament: Game of the Year Edition,Don't Starve: Reign of Giants,The Expendabros,Freedom Planet - Official Soundtrack,Outlast: Whistleblower DLC,The Journey Down: Chapter Two,Doom & Destiny
9,299153.0,Portal 2,Left 4 Dead 2,Counter-Strike: Source,Half-Life 2: Episode Two,Left 4 Dead,Half-Life: Opposing Force,Alien Swarm,Half-Life 2: Episode One,Half-Life: Blue Shift,...,Half-Life 2: Deathmatch,Half-Life: Source,Ricochet,Half-Life Deathmatch: Source,,,,,,


In [197]:
print(gen_pub_dev_matrix)

[[1.         0.2236068  0.25354628 ... 0.         0.         0.12909944]
 [0.2236068  1.         0.47245559 ... 0.         0.         0.14433757]
 [0.25354628 0.47245559 1.         ... 0.         0.         0.10910895]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.12909944 0.14433757 0.10910895 ... 0.         0.         1.        ]]


In [198]:
df_tags_dev_gen, tags_dev_gen_matrix = make_recommendationforNusers('genre_popular_tags_developer')
df_tags_dev_gen

,user_id,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,,,,,,,,,,,...,,,,,,,,,,
1,5250.0,Battlefield: Bad Company 2 Vietnam,BattleBlock Theater®,Serious Sam Classic: The Second Encounter,Left 4 Dead 2,Killing Floor,Counter-Strike: Source,Quake III Arena,QUAKE,Half-Life 2: Episode Two,...,F.E.A.R.,Left 4 Dead,BioShock™,Mass Effect 2,Dead Space™ 2,Sanctum 2,Black Mesa,Half-Life 2: Episode One,Synergy,Call of Duty®
2,76767.0,Battlefield: Bad Company 2 Vietnam,Mount & Blade: Warband,The Expendabros,BattleBlock Theater®,The Witcher® 3: Wild Hunt,Unreal Tournament: Game of the Year Edition,Serious Sam Classic: The Second Encounter,Left 4 Dead 2,Half-Life 2,...,Dishonored,Half-Life 2: Episode Two,Quake III Arena,Counter-Strike: Source,QUAKE,Papo & Yo,Dead Space™ 2,F.E.A.R.,Guacamelee! Super Turbo Championship Edition,Half-Life: Opposing Force
3,86540.0,Portal 2,Battlefield: Bad Company 2 Vietnam,Enigmatis 2: The Mists of Ravenwood,The Room,Mount & Blade: Warband,Grim Legends 2: Song of the Dark Swan,Half-Life 2,Botanicula,The Journey Down: Chapter Two,...,Broforce,Duck Game,Amnesia: The Dark Descent,DARK SOULS™ II - Season Pass,Counter-Strike: Source,Quake III Arena,SOMA,Half-Life 2: Episode Two,QUAKE,Detective Grimoire
4,103360.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Half-Life 2,Counter-Strike: Source,Quake III Arena,Half-Life 2: Episode Two,QUAKE,Left 4 Dead,Half-Life: Opposing Force,...,Alien Swarm,F.E.A.R.,Half-Life 2: Episode One,Team Fortress 2,QUAKE II,Call of Duty: World at War,Descent 2,Call of Duty®,Tom Clancy's Splinter Cell Chaos Theory®,Guns of Icarus Online
5,144736.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Half-Life 2,Counter-Strike: Source,Quake III Arena,Half-Life 2: Episode Two,QUAKE,Left 4 Dead,Mass Effect 2,...,Alien Swarm,Half-Life 2: Episode One,Team Fortress 2,Tom Clancy's Splinter Cell Chaos Theory®,Call of Duty®,Descent 2,Call of Duty: World at War,QUAKE II,Delta Force: Black Hawk Down,Arma 2: Operation Arrowhead
6,181212.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Counter-Strike,QUAKE,Quake III Arena,Half-Life 2: Episode Two,Alien Swarm,Mass Effect 2,F.E.A.R.,...,BioShock™,Half-Life: Opposing Force,Black Mesa,Half-Life 2: Episode One,Team Fortress 2,QUAKE II,Call of Duty®,Descent 2,Call of Duty: World at War,Tom Clancy's Splinter Cell Chaos Theory®
7,229911.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Counter-Strike: Source,Half-Life 2: Episode Two,QUAKE,Quake III Arena,Half-Life: Opposing Force,Left 4 Dead,Mass Effect 2,...,F.E.A.R.,Alien Swarm,Half-Life 2: Episode One,Worms Armageddon,Black Mesa,Call of Duty: World at War,Descent 2,Call of Duty®,Synergy,QUAKE II
8,298950.0,The Witcher 3: Wild Hunt - Expansion Pass,Battlefield: Bad Company 2 Vietnam,Portal 2,Luxor Evolved,Eternal Senia,The Witcher® 3: Wild Hunt,BattleBlock Theater®,Primordia,The Binding of Isaac: Rebirth,...,Outlast: Whistleblower DLC,planetarian ~the reverie of a little planet~,The Expendabros,Enigmatis 2: The Mists of Ravenwood,Escape Goat,Don't Starve: Reign of Giants,Epic Battle Fantasy 4,Botanicula,Hexcells,Freedom Planet
9,299153.0,Battlefield: Bad Company 2 Vietnam,Unreal Tournament: Game of the Year Edition,Counter-Strike: Source,Half-Life 2: Episode Two,Quake III Arena,QUAKE,Mass Effect 2,Alien Swarm,Half-Life: Opposing Force,...,BioShock™,Half-Life 2: Episode One,Black Mesa,Team Fortress 2,Descent 2,Call of Duty®,Tom Clancy's Splinter Cell Chaos Theory®,Call of Duty: World at War,QUAKE II,Return to Castle Wolfenstein


In [199]:
print(tags_dev_gen_matrix)

[[1.         0.44378474 0.53293871 ... 0.12209657 0.         0.24019223]
 [0.44378474 1.         0.68774638 ... 0.08897681 0.         0.21879749]
 [0.53293871 0.68774638 1.         ... 0.02114775 0.         0.20801257]
 ...
 [0.12209657 0.08897681 0.02114775 ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.24019223 0.21879749 0.20801257 ... 0.         0.         1.        ]]
